# **Preprocessing avant utilisation des modèles**

Ce notebook détaille les étapes de preprocessing implémentées et leur justification.

In [1]:
# Pour faciliter la mise à jour des fonctions écrites dans func_custom sans avoir à redémarrer le kernel
%load_ext autoreload
%autoreload 2

In [2]:
# Packages classiques
import pandas as pd
import numpy as np

# Custom package
import func_custom as fc

# NLP
import unicodedata
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem import SnowballStemmer
import spacy

In [3]:
df = pd.read_excel("data/data_train.xlsx",
                    usecols = ["label", "message"])

# 1. Analyse, justification et étapes du preprocessing

Si certaines étapes sont standards et ne nécessite pas d'explication (mettre en .lower() par exemple), regardons certains points plus particulièrement

## 1.1 Application de .lower() : non commenté

## 1.2 Encodage des accents

On a pu constater des problèmes dans le WordCloud du notebook précédent, regardons plus précisément.

In [4]:
# En apparence les mots issus du .xlsx et écrits au clavier se ressemblent
first_message = df["message"][0].split()
data_encodage = [first_message[i] for i in [21, 37, 43, 46]]
print(data_encodage)
keyboard_encodage = ["prélèvement", "époux", "à", "impôts"]
print(keyboard_encodage)

['prélèvement', 'époux', 'à', 'impôts']
['prélèvement', 'époux', 'à', 'impôts']


In [5]:
print("Encodage des accents")
print([t.encode("unicode_escape") for t in data_encodage])
print([t.encode("unicode_escape") for t in keyboard_encodage])
print("\n")
print("Conséquence sur la lemmatization : pas les mêmes racines")
stemmer = SnowballStemmer('french')
print([stemmer.stem(t) for t in data_encodage])
print([stemmer.stem(t) for t in keyboard_encodage])
print("\n")
print("Conséquence sur l'embedding : en dehors du dictionnaire pour spacy !")
nlp = spacy.load("fr_core_news_md")
print([np.sum(nlp(t)[0].vector) for t in data_encodage])
print([np.sum(nlp(t)[0].vector) for t in keyboard_encodage])
print("\n")
print("Encodage après normalisation")
print([unicodedata.normalize("NFKC", t).encode("unicode_escape") for t in data_encodage])
print([unicodedata.normalize("NFKC", t).encode("unicode_escape") for t in keyboard_encodage])
print("\n")
print("Embedding après normalisation : tout rentre dans l'ordre")
print([np.sum(nlp(unicodedata.normalize("NFKC", t))[0].vector) for t in data_encodage])
print([np.sum(nlp(unicodedata.normalize("NFKC", t))[0].vector) for t in keyboard_encodage])

Encodage des accents
[b'pre\\u0301le\\u0300vement', b'e\\u0301poux', b'a\\u0300', b'impo\\u0302ts']
[b'pr\\xe9l\\xe8vement', b'\\xe9poux', b'\\xe0', b'imp\\xf4ts']


Conséquence sur la lemmatization : pas les mêmes racines
['prélèv', 'époux', 'à', 'impôt']
['prélev', 'époux', 'à', 'impôt']


Conséquence sur l'embedding : en dehors du dictionnaire pour spacy !


c:\Users\TitouanBlaize\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\utils\generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
c:\Users\TitouanBlaize\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\utils\generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


[0.0, 0.0, -37.616055, 0.0]
[-0.74289227, 20.48502, -41.42349, 39.59558]


Encodage après normalisation
[b'pr\\xe9l\\xe8vement', b'\\xe9poux', b'\\xe0', b'imp\\xf4ts']
[b'pr\\xe9l\\xe8vement', b'\\xe9poux', b'\\xe0', b'imp\\xf4ts']


Embedding après normalisation : tout rentre dans l'ordre
[-0.74289227, 20.48502, -41.42349, 39.59558]
[-0.74289227, 20.48502, -41.42349, 39.59558]


Il faudra une étape de normalisation avec `unicodedata.normalize("NFKC", text)` sinon l'embedding de Spacy utilisé par la suite retourne des vecteurs nuls.

## 1.3 Gestion de la ponctuation

Les signes de ponctuations sont tous remplacés par un _espace_ pour anticiper sur les failles du tokenizer de nltk. Typiquement en cas d'erreur de phrase, d'absence d'espace, surtout autour de mots inconnus le tokenizer rate complètement, c'est normal il s'agit ici de textes libres écrits par des contribuables et non des textes académiques/de presse. Exemple avec cette phrase qui contient `0000.L'XXXXX́e` :

In [6]:
message_test = df[df["message"].str.contains("0000.L'XXXXX́e")]["message"].values[0]
print(message_test)

Bonjour, J'ai divorcé en 0000.J'ai commencé à verser une pension alimentaire à partir de septembre 0000.L'XXXXX́e dernière j'ai donc versé un XXXXX́e pleine.Par rapport au prélèvement à la source comment cela ce passe-t-il sachant que j'aurais plus à déduire pour l'XXXXX́e dernière que 0000?actuellement je suis prélever par rapport à 0000?y-aura t-il un remboursement en fin d'XXXXX́e?Peut-on modifier le prélèvement à la source en cours d'XXXXX́e? D'avance merci Cordialement XXXXX XXXXX


In [7]:
tokens = word_tokenize(message_test, language = "french")

On constate un problème avec le tokenizer qui persisterait si on supprimait simplement la ponctuation, je préfère donc la remplacer par un espace.

In [8]:
tokens[16]

"0000.L'XXXXX́e"

In [9]:
tokens = word_tokenize(fc.replace_punctuation_with_space(message_test), language = "french")

In [10]:
tokens[18:21]

['0000', 'L', 'XXXXX́e']

## 1.4 Tokenizer

J'utilise ici le tokenizer par défaut de nltk en français

## 1.5 Suppression des caractères de taille 1

Pour prendre en compte des caractères spéciaux ou des lettres uniques suite au tokenizer, qui n'apporteront pas d'informations. Cela permet de limiter la taille des stopwords à inclure. Par ailleurs avec l'encodage diacritique la longueur de `à` serait en réalité de deux.

## 1.6 Gestion des stopwords

En combinant tous les éléments précédents dans la fonction `preprocess_stopwords` dans `func_custom.py` étudions désormais le traitement des stopwords. 
Commençons pas ne prendre en compte que ceux de base dans nltk :

In [11]:
stopwords_french = set(stopwords.words('french'))

In [12]:
text = ' '.join(df["message"].dropna())
tokens_clean = fc.preprocess_stopwords(text, stopwords_french)
pd.DataFrame(tokens_clean).value_counts().head(15)

0           
xxxxx           1777
taux            1323
revenus          712
prélèvement      670
bonjour          624
source           526
cordialement     413
plus             366
merci            365
déclaration      300
comment          271
faire            260
bien             256
situation        255
xxxxx́e          253
Name: count, dtype: int64

On constate que la simple inclusion des stopwords par défaut de nltk ne suffira pas, des éléments spécifiques au jeu de données (`xxxxx` et `0000` issus de pseudonymisation, `bonjour` et `merci` et mots de politesse du fait qu'il s'agit de message écrits par des particuliers à la DGFIP). On peut alors compléter de manière _ad hoc_ cette liste en regardant à l'oeil nu ces éléments. Si l'exercice consiste cependant à distinguer les messages "polis" des autres alors cette liste de stopwords n'est pas pertinente.

TF-IDF aurait dans une certaine mesure pu tenir compte de ces mots largement présent dans le corpus et peu informatif, autant traiter ce problème à la racine ce qui limitera la taille des données à traiter par la suite.

In [13]:
stopwords_adhoc = {"à", "xxxxx", "bonjour", "cordialement", "merci", "xxxxx́e", "xxxxx́", "k€", "donc", "car", "cette", "cela",
                  "être", "si", "même", "faire", "avoir", "remercie", "madame", "monsieur"}
stopwords_complete = stopwords_french.union(stopwords_adhoc)

In [14]:
tokens_clean = fc.preprocess_stopwords(text, stopwords_complete)
pd.DataFrame(tokens_clean).value_counts().head(20)

0            
taux             1323
revenus           712
prélèvement       670
source            526
plus              366
déclaration       300
comment           271
bien              256
situation         255
mois              249
retraite          232
depuis            229
salaire           213
impôt             210
compte            201
avance            201
euros             192
individualisé     183
réponse           181
impôts            180
Name: count, dtype: int64

## 1.7 Prise en compte des fautes d'orthographe : une méthode simple

Je ne vais pas essayer de prendre en compte exhaustivement des fautes d'orthographe mais me contenter d'identifier les erreurs les plus fréquentes dans le corpus. Cela me permettra 1) de corriger les erreurs fréquentes mais utile à garder (par exemple un accent manquant dans prélèvement) et 2) augmenter retrospectivement la liste des stopwords avec des abbréviations non pertinentes (`mme`, `mr`, `svp`).

Important , on constate la présence récurrente du PACS or on peut anticiper qu'il y aura un soucis de vocabulaire lors des méthodes d'embedding.

In [15]:
# Téléchargeons une liste de mots français, par exemple au lien suivant :
#  https://github.com/chrplr/openlexicon/blob/master/datasets-info/Liste-de-mots-francais-Gutenberg/README-liste-francais-Gutenberg.md
with open("data/mots dictionnaires.txt", "r", encoding= "utf-8") as fichier:
   # Lire toutes les lignes du fichier et les stocker dans une liste
   dic_french = set([(unicodedata.normalize("NFKC", ligne.strip().lower())) for ligne in fichier.readlines()])

In [16]:
typo_investiguer = []
for elem in tokens_clean:
    if not elem in dic_french:
        typo_investiguer.append(elem)

In [17]:
pd.DataFrame(typo_investiguer).value_counts().head(50)

0          
pacs           99
mme            87
er             62
impots         55
pacsé          53
xx             51
mr             44
pole           39
pacsée         28
svp            28
impot          27
pacsés         26
aujourd        26
gouv           24
bnc            21
n°             19
cdd            18
prelevement    17
prélévement    17
france         17
carsat         16
aout           16
connaitre      14
cdi            14
etant          14
fr             13
agirc          12
cdt            11
eur            11
xxxxx́s        11
plait          10
rib            10
prélevement    10
xxxxx́es        9
ir              9
ca              9
meme            8
xxxxxe          8
arrco           8
email           8
chomage         8
pls             7
puisqu          7
fip             7
mlle            6
reponse         6
etre            6
infos           6
csg             6
tns             6
Name: count, dtype: int64

In [18]:
stopwords_adhoc = {"à", "xxxxx", "bonjour", "cordialement", "merci", "xxxxx́e", "xxxxx́", "k€", "donc", "car", "cette", "cela",
                  "être", "si", "même", "faire", "avoir", "remercie", "madame", "monsieur",
                  "mme", "mr", "er", "xx", "svp"}
stopwords_complete = stopwords_french.union(stopwords_adhoc)

In [19]:
correction_list = {
    "impot" : "impôt",
    "prélévement" : "prélèvement",
    "prelevement" : "prélèvement",
    "pole" : "pôle"
}

## 1.8 Stemmer

J'utilisais un des stemmer classiques de NLTK, et qui est adapté au Français : SnowballStemmer. Cependant j'ai constaté une légère dégradation des performances du TF-IDF avec stemming, je ne l'utilise finalement pas pour le preprocess et préfère une lemmetizer.

In [20]:
stemmer = SnowballStemmer('french')
# tokens = [stemmer.stem(mot) for mot in tokens]

## 1.9 Lemmetizer

Au lieu du stemming je peux avoir recours au lemmetizer, par exemple celui de Spacy : 

In [ ]:
# nlp = spacy.load("fr_core_news_md")
# tokens = [token.lemma_ for token in nlp(" ".join(tokens))]

# 2. Agrégation dans une fonction unique et application

In [21]:
# Très long à cause de l'ajout du lemmetizer de Spacy (35min au lieu de 1min avant)
df["message_clean"] = df["message"].apply(lambda x : fc.preprocess_text(x, stopwords_complete))

In [22]:
message_test = df.sample(1)
print(message_test["message"].values[0])
print(message_test["message_clean"].values[0])

Bonjour, Selon le taux individualisé qui m'est alloué pour le prélèvement à la source, je ne sais pas combien devra être m'être prélevé chaque mois. Le montant prélevé sera-t-il variable ou restera t-il constant? De plus, que faisons nous des factures relatives à la garde des enfants pour 0000 et 0000? Je vous remercie par avance de votre réponse.
selon taux individualisé allouer prélèvement source savoir combien devoir prélever chaque mois montant prélever variable rester constant plus faire facture relatif garde enfant avancer réponse


In [24]:
df.to_csv("data/data_clean.csv", 
          sep = ";",
          index = False)

# Autre possibilité de _serialization_
# df.to_pickle("data/data_clean.pkl")